# Sentiment Analysis: Classifying Amazon Healthcare Product Reviews 


 




In [1]:
%%time

import math
import re
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from matplotlib.mlab import PCA as mlabPCA
from sklearn.naive_bayes import BernoulliNB
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn.utils import resample
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import KernelPCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

from datetime import datetime
from dateutil.parser import parse
from nltk.stem.porter import PorterStemmer




# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="sklearn"  
    )

# Set Plot Style
sns.set_style('white')

CPU times: user 1.58 s, sys: 476 ms, total: 2.05 s
Wall time: 1.63 s


In [2]:
from sklearn.utils.extmath import randomized_svd
import scipy

In [3]:
%%time

## Import Files

df = pd.read_json('Health_and_Personal_Care_5.json', lines=True)



CPU times: user 4.56 s, sys: 1.08 s, total: 5.64 s
Wall time: 5.7 s


In [4]:
## View Data for Cleaning

#df.head(7)
#df.dtypes
#df.describe()
#df.isnull().sum(axis = 0)
#len(df)

In [5]:
%%time

## Drop Unnecessary Columns

df=df.drop('reviewerName', axis=1)
df=df.drop('reviewTime', axis=1)

## Separate Original Dataset into Separate DataFrames for Different Cleaning Implementations

df_without_text = df.drop(['reviewText', 'summary', 'reviewerID', 'asin'], axis=1)
df_review_text = df.reviewText
df_review_summary = df.summary


CPU times: user 196 ms, sys: 8.49 ms, total: 204 ms
Wall time: 203 ms


In [6]:
%%time

## Converting Helpfulness Rating to float

df_without_text.helpful = df_without_text.helpful.apply(lambda x: str(x).replace('[','').replace(', ','/').replace(']',''))

df_without_text.helpful = df_without_text.helpful.replace('0/0','0')

df_without_text.helpful = df_without_text.helpful.apply(eval)

CPU times: user 3.16 s, sys: 149 ms, total: 3.31 s
Wall time: 3.33 s


In [7]:
%%time

## Vectorizing Text Data

porter_stemmer = PorterStemmer()

def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

tfidf_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=stemming_tokenizer, use_idf=True)
X = tfidf_vectorizer.fit_transform(df_review_text)
df_review_text_vectorized = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names())

tfidf_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=stemming_tokenizer, use_idf=True)
X = tfidf_vectorizer.fit_transform(df_review_summary)
df_review_summary_vectorized = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names())

CPU times: user 12min, sys: 29.9 s, total: 12min 30s
Wall time: 12min 37s


In [ ]:
#np_review_text_vectorized = scipy.sparse.csr_matrix(df_review_text_vectorized)

In [ ]:
## Dimension reduction

#svd= randomized_svd(df_review_text_vectorized, n_components=200)


In [ ]:
## Dimension reduction

svd= TruncatedSVD(200)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
df_review_summary_components = lsa.fit_transform(df_review_text_vectorized)

In [ ]:
variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

In [ ]:
len(df_review_text_vectorized.columns)

In [ ]:
type(eval('0'))

In [ ]:
datetime.fromtimestamp(1329523200).timestamp()

## Data Cleaning Plan

create 3 separate dataframes one with just the review text, one with everything else, one with the summary

Clean Other variables: <br>
Turn helpfulness review into a fraction <br>
Convert overall to positive or negative (4 or 5 is positive) <br>

convert unix review time (maybe just for analysis)

Vectorize review text <br>

Vectorize summary text

Concatenate

In [ ]:
df.head()

In [ ]:
df.dtypes
#df.describe()
#df.isnull().sum(axis = 0)

In [ ]:
#df.dtypes
#df.describe()
df.isnull().sum(axis = 0)

Data exploration review id: if review id are few and far enough apart it can be dropped from the model (compare value counts to len df)/ drop reviewer name regardless
read reviews as part of the data analysis (query)
Summary tokens were intentionally not unionized with review tokens so that the model could pick up on the differences
debate whether to include product name in actual model
consider additional feature engineering to account for helpfulness rating
debate keeping review id and product id

Include examples of types of products in the data analysis
mention how overall was made/  how data was labeled